# `DataLoaders` for feeding data into models

In [ ]:
from src.setup_logging import *
from vaep.io.datasplits import DataSplits

data = DataSplits.from_folder(folder='data/msinstrument_in_QE4', use_wide_format=True)
data.test_X

## Collab

## Denoising Autoencoder

In [ ]:
from fastai.tabular.core import Normalize
from fastai.tabular.core import FillMissing
from fastai.tabular.core import TabularPandas

In [ ]:
to = TabularPandas(data.train_X, procs=None, cont_names=None)
print("Tabular object:", type(to))

## Variational Autoencoder